In [2]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tslearn.metrics import dtw, soft_dtw, dtw_path
from sklearn.metrics import accuracy_score
from scipy.spatial import distance as dist
from scipy.stats import norm

from reader import get_x_label, get_class_x
from average import amplitude_avg, dtw_avg, soft_dtw_avg

In [56]:
file_name = 'yoga'

In [57]:
class_x = get_class_x(file_name + '_TRAIN')
class_avg = dtw_avg(class_x)

In [58]:
dtw_stat = [[list() for i in range(len(class_x[0][0]))] for _ in range(len(class_x))]
for i in range(class_x.shape[0]):
    for j in range(len(class_x[i])):
        path, dist = dtw_path(class_avg[i], class_x[i][j])
        for avg_pos, x_pos in path:
            dtw_stat[i][avg_pos].append(class_x[i][j][x_pos])
dtw_stat = np.array(dtw_stat)
for i in range(dtw_stat.shape[0]):
    for j in range(dtw_stat.shape[1]):
        dtw_stat[i] = np.array(dtw_stat[i])

In [59]:
dtw_mean = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
dtw_sd = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
for class_label in range(dtw_stat.shape[0]):
    for point in range(dtw_stat.shape[1]):
        stat = np.array(dtw_stat[class_label][point])
        dtw_mean[class_label][point].append(stat.mean())
        dtw_sd[class_label][point].append(stat.std())

In [60]:
test_x, test_label = get_x_label(file_name + '_TEST')

In [61]:
bay_predict = []
dist_predict = []
EPSILON = 10**(-10)
count = 0
for x in test_x:
    min_log_prob = 10**(310)
    min_dist = 100000
    bay_nearest_class = -1
    dist_nearest_class = -1
    for j in range(len(class_avg)):
        # Class j
        path, dist = dtw_path(x, class_avg[j])
        log_prob = 1
        probs = [list() for _ in range(len(x))]
        for x_pos, avg_pos in path:
            area_up = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).cdf(x[x_pos])
            area_low = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).cdf(x[x_pos] - EPSILON)
            prob = area_up - area_low
            if prob < 10**(-310):
                prob = 10**(-310)
            probs[x_pos].append(prob)
        for pos_probs in probs:
#             print(len(pos_probs))
            max_pos_prob = max(pos_probs)
#             print(max_pos_prob)
            log_prob += -np.log(max_pos_prob)
#         print(log_prob)
        if log_prob < min_log_prob:
            min_log_prob = log_prob
            bay_nearest_class = j
        if dist < min_dist:
            min_dist = dist
            dist_nearest_class = j
    bay_predict.append(bay_nearest_class)
    dist_predict.append(dist_nearest_class)
    print(count, ':', bay_nearest_class, dist_nearest_class, test_label[count]-1)
    count += 1

0 : 0 1 0
1 : 0 1 1
2 : 0 0 0
3 : 0 0 1
4 : 0 0 0
5 : 0 1 0
6 : 1 1 1
7 : 0 0 0
8 : 0 1 0
9 : 0 0 1
10 : 0 0 0
11 : 0 0 1
12 : 0 0 1
13 : 1 1 1
14 : 0 0 0
15 : 0 0 0
16 : 0 0 0
17 : 0 1 1
18 : 0 0 0
19 : 0 0 1
20 : 0 1 0
21 : 1 1 1
22 : 0 1 1
23 : 0 0 0
24 : 0 0 1
25 : 0 1 0
26 : 0 0 0
27 : 0 0 1
28 : 0 0 0
29 : 1 1 1
30 : 0 1 1
31 : 0 1 0
32 : 0 1 1
33 : 0 0 1
34 : 0 1 0
35 : 0 1 0
36 : 0 1 1
37 : 0 0 0
38 : 0 1 0
39 : 0 1 1
40 : 1 1 1
41 : 0 0 1
42 : 0 0 0
43 : 1 1 1
44 : 0 1 1
45 : 0 0 1
46 : 0 0 1
47 : 0 0 1
48 : 0 1 1
49 : 1 0 1
50 : 1 1 1
51 : 0 1 0
52 : 0 0 0
53 : 1 1 1
54 : 0 1 0
55 : 1 1 1
56 : 0 1 1
57 : 0 0 0
58 : 0 1 0
59 : 0 1 0
60 : 0 1 0
61 : 0 0 1
62 : 1 1 1
63 : 0 0 1
64 : 0 0 0
65 : 0 1 1
66 : 1 1 0
67 : 0 1 1
68 : 0 0 0
69 : 1 0 1
70 : 0 0 1
71 : 0 0 0
72 : 0 0 1
73 : 0 0 0
74 : 0 0 0
75 : 0 0 0
76 : 0 0 1
77 : 0 1 0
78 : 0 0 1
79 : 0 1 1
80 : 0 1 0
81 : 0 0 0
82 : 1 0 1
83 : 0 1 1
84 : 0 0 1
85 : 1 1 1
86 : 0 1 0
87 : 1 1 1
88 : 0 0 0
89 : 0 0 1
90 : 0 0 0
91 : 0 0 

In [62]:
accuracy_score(test_label-1, bay_predict)

0.562